In [ ]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project") 
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###

# Choose a name for your CPU cluster
compute_cluster_name = "compute-cluster"

# Verify that the cluster does not exist already

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os #> es wird in Bild 4 erwähnt

# Specify parameter sampler
#ps = ### YOUR CODE HERE ### -> uniform. 'Tune hyperparmaters for your model with Azure ML'
ps = RandomParameterSampling( {
        "--C": uniform(0.1, 1.0),
        "--max_iter": choice([50, 100, 200])
    }
)

# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=1, slack_factor = 0.1) # early_termination_policy

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder= './training'
os.makedirs(script_folder, exist_ok=True)

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ### 'SKLearn Class' 
import shutil
shutil.copy('./train.py', script_folder)

est = SKLearn(source_directory=script_folder, #kann vielleicht entfallen
              compute_target=compute_cluster,
              script_params={'--C': 1.0, '--max_iter': 100}, #alternatively: ['--C', 1.0, '--max_iter', 100]
              entry_script='train.py') #A string representing the relative path to the file used to start training.

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ### 'Tune hyperparameters for your model with Azure ML'
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=3) #similar to the number of nodes

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)


In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()

#joblib.dump(best_run, 'outputs/SKlearn_hyperdrive.joblib')
mod = best_run.register_model(model_name='SKlearn_hyperdrive', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
dataset = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv', separator=',', header=True, validate=False)
#ds = dataset.to_pandas_dataframe() 


In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

#from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(dataset)
ds = x.join(y)
#todo: Split data into train and test set
x_train_df, x_test_df = train_test_split(ds, test_size=0.2)


In [ ]:
#Convert Dataframe to TabularDataset

from azureml.core import Workspace, Dataset
local_path_train = 'data/prepared_x_train.csv'
local_path_test = 'data/prepared_x_test.csv'
x_train_df.to_csv(local_path_train)
x_test_df.to_csv(local_path_test)

# upload the local file to a datastore on the cloud

subscription_id = 'xxxxxxxxxxxxxxxxxxxxx' 
resource_group = 'xxxxxx'
workspace_name = 'xxxxxxxxxxxxxxxx'

workspace = Workspace(subscription_id, resource_group, workspace_name)

# get the datastore to upload prepared data
datastore = workspace.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='data', target_path='data')

# create a dataset referencing the cloud location
x_train = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/prepared_x_train.csv'))])
x_test = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/prepared_x_test.csv'))])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target
    experiment_timeout_minutes=30,
    task='classification', #logisticRegression is a classification task
    primary_metric='accuracy',
    training_data=x_train, 
    label_column_name='y', #"y"
    n_cross_validations=5)

In [ ]:
!pip install azureml-core
!pip install azureml-explain-model
!pip install --upgrade arzureml-sdk[automl]
!pip install pyopeenssl=16.0.0
!pip install nazureml-core

In [2]:
# Submit your automl run
from azureml.widgets import RunDetails
from azureml.core import Workspace, Experiment
### YOUR CODE HERE ###
ws= Workspace.from_config()
exp= Experiment(workspace=ws, name='udacity-project')

run = exp.submit(automl_config, show_output=True)
#run.wait_for_completion(show_output=True)
#RunDetails(run).show()

In [1]:
# Retrieve and save your best automl model.
import joblib
### YOUR CODE HERE ###

best_automl_run, best_model = run.get_output()
#best_model_run.register_model(model_name='best_run_automl','./outputs')
joblib.dump(value=best_model, filename="best_AutoML.pkl")

#best_automl_model = run.get_best_run_by_primary_metric()
#joblib.dump(best_automl_model, 'AutoML.joblib')

NameError: name 'run' is not defined

In [ ]:
compute_target.delete()

In [ ]:
print(finished)